# 1. Preparation

In [62]:
import pandas as pd
eICU_data = pd.read_csv("C:\\Users\\lee39\\OneDrive\\Desktop\\final_merged_dataset.csv")
cleaned_df = eICU_data.copy()

In [63]:
cleaned_df.drop(["patientunitstayid"], inplace=True, axis=1)
cleaned_df.drop(["hospitalid"], inplace=True, axis=1)

time = cleaned_df["unitdischargeoffset"]
cleaned_df.drop(["unitdischargeoffset"], inplace=True, axis=1)

treatment = cleaned_df["has_Vasopressor"]
cleaned_df.drop(["has_Vasopressor"], inplace=True, axis=1)

event = (cleaned_df['unitdischargestatus'] == 'Expired').astype(int)
cleaned_df.drop(["unitdischargestatus"], inplace=True, axis=1)

categorical_features = ["gender",
                    "ethnicity",
                    "admission_type",
                    "has_COPD",
                    "has_Diabetes",
                    "has_Metastasis",
                    "has_Sepsis_A41_9"]

numerical_features = list(cleaned_df.columns.drop(categorical_features))


In [64]:
treatment

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
35484    0.0
35485    0.0
35486    0.0
35487    0.0
35488    0.0
Name: has_Vasopressor, Length: 35489, dtype: float64

In [65]:
print(f"Categorical features: {categorical_features}")
print(f"Numerical features: {numerical_features}")

Categorical features: ['gender', 'ethnicity', 'admission_type', 'has_COPD', 'has_Diabetes', 'has_Metastasis', 'has_Sepsis_A41_9']
Numerical features: ['age', 'GCS', 'vent', 'WBC x 1000', 'Hgb', 'platelets x 1000', 'total bilirubin', 'creatinine', 'albumin', 'anion gap']


In [14]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35489 entries, 0 to 35488
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            35489 non-null  object 
 1   age               35489 non-null  int64  
 2   ethnicity         35489 non-null  object 
 3   admission_type    35489 non-null  object 
 4   has_COPD          35489 non-null  float64
 5   has_Diabetes      35489 non-null  float64
 6   has_Metastasis    35489 non-null  float64
 7   has_Sepsis_A41_9  35489 non-null  float64
 8   GCS               35489 non-null  float64
 9   vent              35489 non-null  float64
 10  WBC x 1000        35489 non-null  float64
 11  Hgb               35489 non-null  float64
 12  platelets x 1000  35489 non-null  float64
 13  total bilirubin   35489 non-null  float64
 14  creatinine        35489 non-null  float64
 15  albumin           35489 non-null  float64
 16  anion gap         35489 non-null  float6

# 2. DCM on eICU

In [66]:
features = pd.concat([cleaned_df, treatment], axis=1)
outcomes = pd.concat([time, event], axis=1)
outcomes.columns = ["time", "event"]

In [67]:
from auton_survival.preprocessing import Preprocessor

features = Preprocessor().fit_transform(features,
                                        cat_feats=categorical_features + ["has_Vasopressor"],
                                        num_feats=numerical_features)

In [91]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35489 entries, 0 to 35488
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   age                        35489 non-null  float64
 1   GCS                        35489 non-null  float64
 2   vent                       35489 non-null  float64
 3   WBC x 1000                 35489 non-null  float64
 4   Hgb                        35489 non-null  float64
 5   platelets x 1000           35489 non-null  float64
 6   total bilirubin            35489 non-null  float64
 7   creatinine                 35489 non-null  float64
 8   albumin                    35489 non-null  float64
 9   anion gap                  35489 non-null  float64
 10  gender_Male                35489 non-null  uint8  
 11  gender_Unknown             35489 non-null  uint8  
 12  ethnicity_Asian            35489 non-null  uint8  
 13  ethnicity_Caucasian        35489 non-null  uin

In [68]:
from auton_survival.preprocessing import Preprocessor
import numpy as np

horizons = [0.25, 0.5, 0.75]
times = np.quantile(outcomes.time[outcomes.event==1], horizons).tolist()

In [69]:
x, t, e = features.values, outcomes.time.values, outcomes.event.values

n = len(x)

tr_size = int(n * 0.70)
vl_size = int(n * 0.10)
te_size = int(n * 0.20)

x_train, x_test, x_val = x[:tr_size], x[-te_size:], x[tr_size:tr_size+vl_size]
t_train, t_test, t_val = t[:tr_size], t[-te_size:], t[tr_size:tr_size+vl_size]
e_train, e_test, e_val = e[:tr_size], e[-te_size:], e[tr_size:tr_size+vl_size]

t = outcomes["time"]
e = outcomes["event"]
quantiles = [0.25, 0.5, 0.75]
quantiles = np.quantile(t[e == 1], quantiles)
quantiles

array([1603., 3921., 8849.])

In [70]:
from sklearn.model_selection import ParameterGrid

param_grid = {'k' : [3],
              'learning_rate' : [1e-3],
              'layers' : [[50, 50] ]
             }
params = ParameterGrid(param_grid)

In [72]:
from auton_survival.models.dcm import DeepCoxMixtures

model = DeepCoxMixtures()

# The fit method is called to train the model
model.fit(x_train, t_train, e_train, iters=100, val_data=(x_val, t_val, e_val))

  0%|          | 0/100 [00:00<?, ?it/s]C:\My_Programs\Anaconda\envs\CSMDRMP\lib\site-packages\auton_survival\models\dcm\dcm_utilities.py:105: RuntimeWarning: invalid value encountered in log
  probs = gates+np.log(event_probs)
C:\My_Programs\Anaconda\envs\CSMDRMP\lib\site-packages\auton_survival\models\dcm\dcm_utilities.py:105: RuntimeWarning: divide by zero encountered in log
  probs = gates+np.log(event_probs)
  6%|▌         | 6/100 [00:18<04:45,  3.04s/it]


In [95]:
train_latent = model.predict_latent_z(x_train)

train_group = np.argmax(train_latent, axis=1)

(24842,)

In [73]:
# plot for different groups

